## The System:

![grafik.png][def]


[def]: .\PTG_System.png

## Preperation
#### Welche Packages brauchen wir NICHT?

In [66]:
!pip install gurobipy==10.0.0
!pip install pyomo==6.6.2
!pip install numpy==1.24.3
!pip install pandas==1.3.5
#!pip install tsam==2.3.1
!pip install matplotlib
#!pip install july

In [67]:
from gurobipy import *
from pyomo.environ import *
#import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#import tsam.timeseriesaggregation as tsam
#import july
#from july.utils import date_range


## Time series
#### Daten laden

In [68]:
data_sheet = pd.read_excel(".\Data_Sheet_neu.xlsx")
print(data_sheet)

time_slices = np.array(data_sheet.index)
#print(time_slices)


time_series = list(range(0, 8760))
next_time_slices = {time_series[i]: time_series[(i + 1) % len(time_series)] for i in range(len(time_series))}
#print(next_time_slice)


# gas price of each time slice
power_demand_time_series = dict(zip(time_slices, np.array(data_sheet['PowerD_demand'])))

# heat demand of each time slice
heat_demand_time_series = dict(zip(time_slices, np.array(data_sheet['HeatD_demand'])))

# electricity_price of each time slice
electricity_price_time_series = dict(zip(time_slices, np.array(data_sheet['electricity_price'])))

# PV_Power of eacj time slice
power_PV_time_series = dict(zip(time_slices, np.array(data_sheet['P_PV_tot'])))



      Unnamed: 0  s     t  dt  PowerD_demand  HeatD_demand  P_PV_tot  \
0              0  1     1   1          158.4     69.178754   0.00000   
1              1  1     2   1          156.0     56.480866   0.00000   
2              2  1     3   1          156.0     45.267146   0.00000   
3              3  1     4   1          156.6     39.165564   0.00000   
4              4  1     5   1          151.2     40.567279   0.00000   
...          ... ..   ...  ..            ...           ...       ...   
8755        8755  1  8756   1          168.6     15.307163   4.33854   
8756        8756  1  8757   1          174.0     14.920863   0.00000   
8757        8757  1  8758   1          167.4     14.293124   0.00000   
8758        8758  1  8759   1          166.8     23.033177   0.00000   
8759        8759  1  8760   1          160.2     32.497543   0.00000   

      electricity_price  
0             -0.013287  
1             -0.002750  
2             -0.003778  
3             -0.013056  
4    

## Setting up the model:

In [69]:
model = ConcreteModel()

# declare sets
model.time_slices = set(time_slices)
model.next_time_slices = set(next_time_slices)

In [70]:
# declare decision variables
model.OPEX = Var(domain=NonNegativeReals) # [€]

# declare time dependent decision variables
model.P_e_buy = Var(model.time_slices, domain=NonNegativeReals) # Power, that needs to get bought from the grid [kW_e]
model.P_CH4_sell = Var(model.time_slices, domain=NonNegativeReals) # CH4, that gets to sell to the grid [kW_CH4]
model.mdot_CO2_atm = Var(model.time_slices, domain=NonNegativeReals) # CH4, that gets to sell to the grid [kW_CH4]

# Component_Inputs
model.P_e_PEM = Var(model.time_slices, domain=NonNegativeReals) # Electrolysis [kW_e]
model.P_e_DAC = Var(model.time_slices, domain=NonNegativeReals) # DirectAirCapture [kW_e]
model.P_H2_MR = Var(model.time_slices, domain=NonNegativeReals) # MethaneReformation [kW_H2]
model.P_CO2_MR = Var(model.time_slices, domain=NonNegativeReals) # MethaneReformation [kW_CO2] STIMMT EINHEIT???
model.P_CH4_CHP = Var(model.time_slices, domain=NonNegativeReals) # CombinedHeatPower [kW_CH4]
model.P_CH4_BO = Var(model.time_slices, domain=NonNegativeReals) # Boiler [kW_CH4]

model.y_PEM = Var(model.time_slices, domain=Binary) # Electrolysis, on/off
model.y_CHP = Var(model.time_slices, domain=Binary) # CombinedHeatPower, on/off
model.y_BO = Var(model.time_slices, domain=Binary) # Boiler, on/off

# Component_Outputs
model.P_e_PV = Var(model.time_slices, domain=NonNegativeReals) # PV [kW_e]
model.P_H2_PEM = Var(model.time_slices, domain=NonNegativeReals) # Electrolysis [kW_H2]
model.mdot_CO2_DAC = Var(model.time_slices, domain=NonNegativeReals) # DirectAirCapture [kg_CO2/hr]
model.P_CH4_MR = Var(model.time_slices, domain=NonNegativeReals) # MethaneReformation [kW_CH4]
model.P_e_CHP = Var(model.time_slices, domain=NonNegativeReals) # CombinedHeatPower [kW_e]
model.P_h_CHP = Var(model.time_slices, domain=NonNegativeReals) # CombinedHeatPower [kW_h]
model.P_h_BO = Var(model.time_slices, domain=NonNegativeReals) # Boiler [kW_h]

# Storages
model.P_e_battery = Var(model.time_slices, domain=Reals) # [negative: storage charge, positive: storage discharge] [kW_e] 
model.P_e_battery_in = Var(model.time_slices, domain=NonNegativeReals) # [kW_e] 
model.P_e_battery_out = Var(model.time_slices, domain=NonNegativeReals) #  [kW_e]
model.storage_level_battery = Var(model.time_slices, domain=NonNegativeReals) # [kWh]
model.in_battery =  Var(model.time_slices, domain=Binary) #einspeichern
model.out_battery = Var(model.time_slices, domain=Binary) #ausspeichern

model.P_h_thermalStorage = Var(model.time_slices, domain=Reals) # [negative: storage charge, positive: storage discharge] [kW_h]
model.P_h_thermalStorage_in = Var(model.time_slices, domain=NonNegativeReals) # [kW_h]
model.P_h_thermalStorage_out = Var(model.time_slices, domain=NonNegativeReals) # [kW_h]
model.storage_level_thermalStorage = Var(model.time_slices, domain=NonNegativeReals) # [kWh]
model.in_thermalStorage =  Var(model.time_slices, domain=Binary) #einspeichern
model.out_thermalStorage = Var(model.time_slices, domain=Binary) #ausspeichern

model.P_H2_gasStorage = Var(model.time_slices, domain=Reals) # [negative: storage charge, positive: storage discharge] [kW_H2]
model.P_H2_gasStorage_in = Var(model.time_slices, domain=NonNegativeReals) # [kW_H2]
model.P_H2_gasStorage_out = Var(model.time_slices, domain=NonNegativeReals) # [kW_H2]
model.storage_level_H2Storage = Var(model.time_slices, domain=NonNegativeReals) # [kWh_H2]
model.in_H2Storage =  Var(model.time_slices, domain=Binary) #einspeichern
model.out_H2Storage = Var(model.time_slices, domain=Binary) #ausspeichern

model.P_CO2_gasStorage = Var(model.time_slices, domain=Reals) # [negative: storage charge, positive: storage discharge] [kg_CO2/hr]
model.P_CO2_gasStorage_in = Var(model.time_slices, domain=NonNegativeReals) # [kg_CO2/hr]
model.P_CO2_gasStorage_out = Var(model.time_slices, domain=NonNegativeReals) # [kg_CO2/hr]
model.storage_level_CO2Storage = Var(model.time_slices, domain=NonNegativeReals) # [kg_CO2]
model.in_CO2Storage =  Var(model.time_slices, domain=Binary) #einspeichern
model.out_CO2Storage = Var(model.time_slices, domain=Binary) #ausspeichern

model.P_CH4_gasStorage = Var(model.time_slices, domain=Reals) # [negative: storage charge, positive: storage discharge] [kW_CH4]
model.P_CH4_gasStorage_in = Var(model.time_slices, domain=NonNegativeReals) # [kW_CH4]
model.P_CH4_gasStorage_out = Var(model.time_slices, domain=NonNegativeReals) # [kW_CH4]
model.storage_level_CH4Storage = Var(model.time_slices, domain=NonNegativeReals) # [kWh_CH4]
model.in_CH4Storage =  Var(model.time_slices, domain=Binary) #einspeichern
model.out_CH4Storage = Var(model.time_slices, domain=Binary) #ausspeichern

In [71]:
# declare parameters

model.price_co2 = 84.4 # [€/ton]
model.price_gas = 0.0775 # [€/kWh]


model.P_e_N_PV = 900 #[kW_e]

#model.P_e_N_PEM = 900 #[kW_e]
#model.eta_N_PEM = 0.63 #eta muss noch linerarisiert werden!!
model.P_H2_min_PEM = 146.75 #[kW_H2]
model.P_H2_max_PEM = 641.13 #[kW_H2]
model.P_e_min_PEM = 180 #[kW_e]
model.P_e_max_PEM = 1080 #[kW_e]
#model.p_min_PEM = 0.2
#model.p_max_PEM = 1.2

model.mdot_CH4_N_DAC = 250 #[kg_CO2/hr]
model.eta_DAC = 0.5 #[kg_CO2/kWh_e]

model.P_CH4_N_MR = 1500 #[kW_CH4]
model.eta_MR = 0.78
model.co2need_MR = 0.178 #E_in_CO2/E_out_CH4


#model.P_h_N_CHP = 800 #[kW_h]
#model.eta__N_CHP_Q = 0.47 #eta muss noch linerarisiert werden!!
model.P_h_min_CHP = 56.525 #[kW_h]
model.P_h_min_CHP = 176.71 #[kW_h]
model.P_e_min_CHP = 36.8 #[kW_e]
model.P_e_max_CHP = 128 #[kW_e]
model.P_CH4_min_CHP = 400 #[kW_CH4]
model.P_CH4_max_CHP = 800 #[kW_CH4]
#model.eta_N_CHP_P = 0.4 #eta muss noch linerarisiert werden!!
#model.p_min_CHP = 0.5
#model.p_max_CHP = 1

#model.P_h_N_BO = 500 #[kW_h]
#model.eta_N_BO = 0.9 #eta muss noch linerarisiert werden!
model.P_CH4_min_BO = 0.5 #[kW_CH4]
model.P_CH4_max_BO = 500 #[kW_cH4]
model.P_h_min_BO = 0.00164#[kW_h]
model.P_h_max_BO = 450 #[kW_h]
#model.p_min_BO = 0.001
#model.p_max_BO = 1

model.E_N_battery = 5000 #[kWh_e]
model.eta_battery = 0.92 
model.loss_battery = 0.000042 #[1/hr]
model.c_battery = 0.36 #[1/hr]

model.Q_N_thermalStorage = 5000 #[kWh_Q]
model.eta_thermalStorage = 0.95 
model.loss_thermalStorage = 0.005 #[1/hr]
model.c_thermalStorage = 1 #[1/hr]

model.E_N_H2Storage = 5000 #[kWh_H2]
model.m_N_CO2Storage = 5000 #[kg_CO2]
model.E_N_CH4Storage = 5000 #[kWh_CH4]
model.loss_gasTanks = 0 #[1/hr]
model.eta_gasTanks = 1 
model.c_gasTanks = 0.25 #[1/hr]


In [72]:
#declare time dependent parameters
model.P_e_demand = Param(model.time_slices, initialize = power_demand_time_series)
model.Q_demand = Param(model.time_slices, initialize = heat_demand_time_series)

model.electricity_price = Param(model.time_slices, initialize = electricity_price_time_series)

model.P_PV = Param(model.time_slices, initialize = power_PV_time_series)


In [73]:
# declare unindexed constraints
model.OPEX_constraint = Constraint(expr =  model.OPEX == sum(model.electricity_price[time_slice]*model.P_e_buy[time_slice] for time_slice in model.time_slices) 
                                     +  sum(model.price_co2 * model.mdot_CO2_atm[time_slice] for time_slice in model.time_slices)
                                     -  sum(model.price_gas * model.P_CH4_sell[time_slice] for time_slice in model.time_slices)
                                     )

# declare rules to setup indexed constraints
def electricty_balance_constraint_rule(model, time_slice):
    return model.P_e_PV[time_slice] + model.P_e_CHP[time_slice] + model.P_e_battery[time_slice] + model.P_e_buy[time_slice] == model.P_e_demand[time_slice] + model.P_e_PEM[time_slice] + model.P_e_DAC[time_slice]

def heat_balance_constraint_rule(model, time_slice):
    return model.P_h_CHP[time_slice] + model.P_h_BO[time_slice] + model.P_h_thermalStorage[time_slice] == model.Q_demand[time_slice]

def h2_balance_constraint_rule(model, time_slice):
    return model.P_H2_PEM[time_slice] + model.P_H2_gasStorage[time_slice] == model.P_H2_MR[time_slice]

def co2_balance_contraint_rule(model, time_slice):
    return model.mdot_CO2_DAC[time_slice] + model.P_CO2_gasStorage[time_slice] == model.P_CO2_MR[time_slice]
# gesmischte EINHEITEN !!!ACHTUNG

def ch4_balance_constraint_rule(model, time_slice):
    return model.P_CH4_MR[time_slice] + model.P_CH4_gasStorage[time_slice] == model.P_CH4_MR[time_slice] + model.P_CH4_BO[time_slice] + model.P_CH4_sell[time_slice]

##Components, Input->Output, Nenngröße, 
def PV_constraint_rule(model, time_slice):
    return model.P_e_PV[time_slice] <= model.P_e_N_PV * model.P_PV[time_slice]
#!!!! passt die Gleichung zu den Inputs? CHecken!!

def PEM_constraint_rule(model, time_slice):
    return model.P_H2_PEM == 47.877 + 0.549 * model.P_e_PEM
def PEM_minInput_constraint_rule(model, time_slice):
    return model.P_e_PEM >= model.P_e_min_PEM * model.y_PEM
def PEM_maxInput_constraint_rule(model, time_slice):
    return model.P_e_PEM <= model.P_e_max_PEM * model.y_PEM
def PEM_minOuput_constraint_rule(model, time_slice):
    return model.P_H2_PEM >= model.P_H2_min_PEM * model.y_PEM #ein/aus Variabel hier notwendig?
def PEM_maxOuput_constraint_rule(model, time_slice):
    return model.P_H2_PEM <= model.P_H2_max_PEM * model.y_PEM #ein/aus Variabel hier notwendig?

def DAC_constraint_rule(model, time_slice):
    return model.mdot_CO2_DAC == model.eta_DAC * model.P_e_DAC
def DAC_maxOuput_constraint_rule(model, time_slice):
    return model.mdot_CO2_DAC <= model.P_mdot_CH4_N_DAC

def MR_constraint_rule(model, time_slice):
    return model.P_CH4_MR == model.eta_MR * model.P_H2_MR
def MR_CO2_constraint_rule(model, time_slice):
    return model.P_CO2_MR == model.co2need_MR * model.P_CH4_MR ##iwie stimmen hier die Einheiten nicht !!!
def MR_maxOuput_constraint_rule(model, time_slice):
    return model.P_CH4_PEM <= model.P_CH4_N_MR

def CHP_heat_constraint_rule(model, time_slice):
    return model.P_h_CHP == -80.948 + 0.317 * model.P_CH4_CHP
def CHP_heat_minOuput_constraint_rule(model, time_slice):
    return model.P_h_CHP >= model.P_h_min_CHP * model.y_CHP #ein/aus Variabel hier notwendig?
def CHP_heat_maxOuput_constraint_rule(model, time_slice):
    return model.P_h_CHP <= model.P_h_max_CHP * model.y_CHP #ein/aus Variabel hier notwendig?
def CHP_electricity_constraint_rule(model, time_slice):
    return model.P_e_CHP == -58.168 + 0.229 * model.P_CH4_CHP
def CHP_electricity_minOuput_constraint_rule(model, time_slice):
    return model.P_e_CHP >= model.P_e_min_CHP * model.y_CHP #ein/aus Variabel hier notwendig?
def CHP_electricity_maxOuput_constraint_rule(model, time_slice):
    return model.P_e_CHP <= model.P_e_max_CHP * model.y_CHP #ein/aus Variabel hier notwendig?
def CHP_minInput_constraint_rule(model, time_slice):
    return model.P_CH4_CHP >= model.P_CH4_min_CHP * model.y_CHP 
def CHP_maxInput_constraint_rule(model, time_slice):
    return model.P_CH4_CHP <= model.P_CH4_max_CHP * model.y_CHP
    
def BO_constraint_rule(model, time_slice):
    return model.P_h_BO  == -26.704 + 0.955 * model.P_CH4_BO
def BO_minInput_constraint_rule(model, time_slice):
    return model.P_CH4_BO >= model.P_CH4_min_BO * model.y_BO
def BO_maxInput_constraint_rule(model, time_slice):
    return model.P_CH4_BO <= model.P_CH4_max_BO * model.y_BO
def PEM_minOuput_constraint_rule(model, time_slice):
    return model.P_h_BO >= model.P_h_min_BO * model.y_BO #ein/aus Variabel hier notwendig?
def PEM_maxOuput_constraint_rule(model, time_slice):
    return model.P_h_BO <= model.P_h_max_BO * model.y_BO #ein/aus Variabel hier notwendig?








##Storages, storage_level, storage_maximum

#Battery
def battery_level_constraint_rule(model, time_slice):
    return (model.storage_level_battery[next_time_slices[time_slice]] == 
        model.storage_level_battery[time_slice] * (1-model.loss_battery)
        + (model.P_e_battery_in[time_slice]*model.eta_battery - model.P_e_battery_out[time_slice]/model.eta_battery))
    
def battery_Energy_balance_constraint_rule(model, time_slice):
    return  (model.P_e_battery[time_slice] == model.P_e_battery_in[time_slice] - model.P_e_battery_out[time_slice])
    
def battery_level_maximum_constraint_rule(model, time_slice):
    return  (model.P_e_battery_in[time_slice] <= model.in_battery[time_slice]*model.c_battery*model.E_N_battery)
    
def battery_level_minimum_constraint_rule(model, time_slice):
    return  (model.P_e_battery_out[time_slice] <= model.out_battery[time_slice]*model.c_battery*model.E_N_battery)

def battery_in_or_out_or_off_constraint_rule(model, time_slice):
    return  (model.in_battery[time_slice] + model.out_battery[time_slice] <= 1)


#thermalStorage
def thermalStorage_level_constraint_rule(model, time_slice):
    return  (model.storage_level_thermalStorage[next_time_slices[time_slice]] == (
        model.storage_level_thermalStorage[time_slice] * (1-model.loss_thermalStorage)
        + (model.P_h_thermalStorage_in[time_slice]*model.eta_thermalStorage - model.P_h_thermalStorage_out[time_slice]/model.eta_thermalStorage))
    )

def thermalStorage_Energy_balance_constraint_rule(model, time_slice):  
    return  (model.P_h_thermalStorage[time_slice] == model.P_h_thermalStorage_in[time_slice] - model.P_h_thermalStorage_out[time_slice])  

def thermalStorage_level_maximum_constraint_rule(model, time_slice):        
    return  (model.P_h_thermalStorage_in[time_slice] <= model.in_thermalStorage[time_slice]*model.c_thermalStorage*model.Q_N_thermalStorage)

def thermalStorage_level_minimum_constraint_rule(model, time_slice):    
    return  (model.P_h_thermalStorage_out[time_slice] <= model.out_thermalStorage[time_slice]*model.c_thermalStorage*model.Q_N_thermalStorage)

def thermalStorage_in_or_out_or_off_constraint_rule(model, time_slice):    
    return  (model.in_thermalStorage[time_slice] + model.out_thermalStorage[time_slice] <= 1)


#H2Storage
def H2Storage_level_constraint_rule(model, time_slice):
    return  (model.storage_level_H2Storage[next_time_slices[time_slice]] == (
        model.storage_level_H2Storage[time_slice] * (1-model.loss_gasTanks)
        + (model.P_H2_gasStorage_in[time_slice]*model.eta_gasTanks - model.P_H2_gasStorage_out[time_slice]/model.eta_gasTanks))
    )   
    
def H2Storage_Energy_balance_constraint_rule(model, time_slice):
    return  (model.P_H2_gasStorage[time_slice] == model.P_H2_gasStorage_in[time_slice] - model.P_H2_gasStorage_out[time_slice]) 

def H2Storage_level_maximum_constraint_rule(model, time_slice):
    return  (model.P_H2_gasStorage_in[time_slice] <= model.in_H2Storage[time_slice]*model.c_gasTanks*model.E_N_H2Storage)

def H2Storage_level_minimum_constraint_rule(model, time_slice):
    return  (model.P_H2_gasStorage_out[time_slice] <= model.out_H2Storage[time_slice]*model.c_gasTanks*model.E_N_H2Storage)

def H2Storage_in_or_out_or_off_constraint_rule(model, time_slice):
    return  (model.in_H2Storage[time_slice] + model.out_H2Storage[time_slice] <= 1)


#CO2Storage
def CO2Storage_level_constraint_rule(model, time_slice):    
    return  (model.storage_level_CO2Storage[next_time_slices[time_slice]] == (
        model.storage_level_CO2Storage[time_slice] * (1-model.loss_gasTanks)
        + (model.P_CO2_gasStorage_in[time_slice]*model.eta_gasTanks - model.P_CO2_gasStorage_out[time_slice]/model.eta_gasTanks))
    )   
    
def CO2Storage_Energy_balance_constraint_rule(model, time_slice):
    return  (model.P_CO2_gasStorage[time_slice] == model.P_CO2_gasStorage_in[time_slice] - model.P_CO2_gasStorage_out[time_slice])  

def CO2Storage_level_maximum_constraint_rule(model, time_slice):
    return  (model.P_CO2_gasStorage_in[time_slice] <= model.in_CO2Storage[time_slice]*model.c_gasTanks*model.m_N_CO2Storage)

def CO2Storage_level_minimum_constraint_rule(model, time_slice):
    return  (model.P_CO2_gasStorage_out[time_slice] <= model.out_CO2Storage[time_slice]*model.c_gasTanks*model.m_N_CO2Storage)

def CO2Storage_in_or_out_or_off_constraint_rule(model, time_slice):
    return  (model.in_CO2Storage[time_slice] + model.out_CO2Storage[time_slice] <= 1)


#CH4Storage	
def CH4Storage_level_constraint_rule(model, time_slice):
    return  (model.storage_level_CH4Storage[next_time_slices[time_slice]] == (
        model.storage_level_CH4Storage[time_slice] * (1-model.loss_gasTanks)
        + (model.P_CH4_gasStorage_in[time_slice]*model.eta_gasTanks - model.P_CH4_gasStorage_out[time_slice]/model.eta_gasTanks))
    )

def CH4Storage_Energy_balance_constraint_rule(model, time_slice):
    return  (model.P_CH4_gasStorage[time_slice] == model.P_CH4_gasStorage_in[time_slice] - model.P_CH4_gasStorage_out[time_slice])

def CH4Storage_level_maximum_constraint_rule(model, time_slice):
    return  (model.P_CH4_gasStorage_in[time_slice] <= model.in_CH4Storage[time_slice]*model.c_gasTanks*model.E_N_CH4Storage)

def CH4Storage_level_minimum_constraint_rule(model, time_slice):
    return  (model.P_CH4_gasStorage_out[time_slice] <= model.out_CH4Storage[time_slice]*model.c_gasTanks*model.E_N_CH4Storage)

def CH4Storage_in_or_out_or_off_constraint_rule(model, time_slice):
    return  (model.in_CH4Storage[time_slice] + model.out_CH4Storage[time_slice] <= 1)


# declare indexed constraints

model.CH4Storage_level_constraint = Constraint(model.time_slices, rule=CH4Storage_level_constraint_rule)
model.CH4Storage_Energy_balance_constraint = Constraint(model.time_slices, rule=CH4Storage_Energy_balance_constraint_rule)
model.CH4Storage_level_maximum_constraint = Constraint(model.time_slices, rule=CH4Storage_level_maximum_constraint_rule)
model.CH4Storage_level_minimum_constraint = Constraint(model.time_slices, rule=CH4Storage_level_minimum_constraint_rule)
model.CH4Storage_in_or_out_or_off_constraint = Constraint(model.time_slices, rule=CH4Storage_in_or_out_or_off_constraint_rule)

model.Co2Storage_level_constraint = Constraint(model.time_slices, rule=CO2Storage_level_constraint_rule)
model.Co2Storage_Energy_balance_constraint = Constraint(model.time_slices, rule=CO2Storage_Energy_balance_constraint_rule)
model.Co2Storage_level_maximum_constraint = Constraint(model.time_slices, rule=CO2Storage_level_maximum_constraint_rule)
model.Co2Storage_level_minimum_constraint = Constraint(model.time_slices, rule=CO2Storage_level_minimum_constraint_rule)
model.Co2Storage_in_or_out_or_off_constraint = Constraint(model.time_slices, rule=CO2Storage_in_or_out_or_off_constraint_rule)

model.H2Storage_level_constraint = Constraint(model.time_slices, rule=H2Storage_level_constraint_rule)
model.H2Storage_Energy_balance_constraint = Constraint(model.time_slices, rule=H2Storage_Energy_balance_constraint_rule)
model.H2Storage_level_maximum_constraint = Constraint(model.time_slices, rule=H2Storage_level_maximum_constraint_rule)
model.H2Storage_level_minimum_constraint = Constraint(model.time_slices, rule=H2Storage_level_minimum_constraint_rule)
model.H2Storage_in_or_out_or_off_constraint = Constraint(model.time_slices, rule=H2Storage_in_or_out_or_off_constraint_rule)

model.thermalStorage_level_constraint = Constraint(model.time_slices, rule=thermalStorage_level_constraint_rule)
model.thermalStorage_Energy_balance_constraint = Constraint(model.time_slices, rule=thermalStorage_Energy_balance_constraint_rule)
model.thermalStorage_level_maximum_constraint = Constraint(model.time_slices, rule=thermalStorage_level_maximum_constraint_rule)
model.thermalStorage_level_minimum_constraint = Constraint(model.time_slices, rule=thermalStorage_level_minimum_constraint_rule)
model.thermalStorage_in_or_out_or_off_constraint = Constraint(model.time_slices, rule=thermalStorage_in_or_out_or_off_constraint_rule)

model.battery_level_constraint = Constraint(model.time_slices, rule=battery_level_constraint_rule)
model.battery_Energy_balance_constraint = Constraint(model.time_slices, rule=battery_Energy_balance_constraint_rule)
model.battery_level_maximum_constraint = Constraint(model.time_slices, rule=battery_level_maximum_constraint_rule)
model.battery_level_minimum_constraint = Constraint(model.time_slices, rule=battery_level_minimum_constraint_rule)
model.battery_in_or_out_or_off_constraint = Constraint(model.time_slices, rule=battery_in_or_out_or_off_constraint_rule)

model.electricty_balance_constraint = Constraint(model.time_slices, rule=electricty_balance_constraint_rule)
model.heat_balance_constraint = Constraint(model.time_slices, rule=heat_balance_constraint_rule)
model.h2_balance_constraint = Constraint(model.time_slices, rule=h2_balance_constraint_rule)
model.co2_balance_contraint = Constraint(model.time_slices, rule=co2_balance_contraint_rule)
model.ch4_balance_constraint = Constraint(model.time_slices, rule=ch4_balance_constraint_rule)

model.PV_constraint = Constraint(model.time_slices, rule=PV_constraint_rule)
model.PEM_constraint = Constraint(model.time_slices, rule=PEM_constraint_rule)
model.PEM_minInput_constraint = Constraint(model.time_slices, rule=PEM_minInput_constraint_rule)
model.PEM_maxInput_constraint = Constraint(model.time_slices, rule=PEM_maxInput_constraint_rule)
model.PEM_minOuput_constraint = Constraint(model.time_slices, rule=PEM_minOuput_constraint_rule)
model.PEM_maxOuput_constraint = Constraint(model.time_slices, rule=PEM_maxOuput_constraint_rule)
model.DAC_constraint = Constraint(model.time_slices, rule=DAC_constraint_rule)
model.MR_constraint = Constraint(model.time_slices, rule=MR_constraint_rule)
model.MR_CO2_constraint = Constraint(model.time_slices, rule=MR_CO2_constraint_rule)
model.MR_maxOuput_constraint = Constraint(model.time_slices, rule=MR_maxOuput_constraint_rule)
model.CHP_heat_constraint = Constraint(model.time_slices, rule=CHP_heat_constraint_rule)
model.CHP_heat_minOuput_constraint = Constraint(model.time_slices, rule=CHP_heat_minOuput_constraint_rule)
model.CHP_heat_maxOuput_constraint = Constraint(model.time_slices, rule=CHP_heat_maxOuput_constraint_rule)
model.CHP_electricity_constraint = Constraint(model.time_slices, rule=CHP_electricity_constraint_rule)
model.CHP_electricity_minOuput_constraint = Constraint(model.time_slices, rule=CHP_electricity_minOuput_constraint_rule)
model.CHP_electricity_maxOuput_constraint = Constraint(model.time_slices, rule=CHP_electricity_maxOuput_constraint_rule)
model.CHP_minInput_constraint = Constraint(model.time_slices, rule=CHP_minInput_constraint_rule)
model.CHP_maxInput_constraint = Constraint(model.time_slices, rule=CHP_maxInput_constraint_rule)
model.BO_constraint = Constraint(model.time_slices, rule=BO_constraint_rule)
model.BO_minInput_constraint = Constraint(model.time_slices, rule=BO_minInput_constraint_rule)
model.BO_maxInput_constraint = Constraint(model.time_slices, rule=BO_maxInput_constraint_rule)
model.PEM_minOuput_constraint = Constraint(model.time_slices, rule=PEM_minOuput_constraint_rule)
model.PEM_maxOuput_constraint = Constraint(model.time_slices, rule=PEM_maxOuput_constraint_rule)


ERROR: Rule failed when generating expression for Constraint PEM_constraint
with index 0: TypeError: unsupported operand type(s) for *: 'float' and
'IndexedVar'
ERROR: Constructing component 'PEM_constraint' from data=None failed:
TypeError: unsupported operand type(s) for *: 'float' and 'IndexedVar'


TypeError: unsupported operand type(s) for *: 'float' and 'IndexedVar'

## Solving the problem

In [ ]:
#SolverFactory('gurobi').solve(model).write()